In [1]:
#Declaring dependencies
from splinter import Browser
from bs4 import BeautifulSoup
from splinter.exceptions import ElementDoesNotExist
import requests
import pandas as pd


In [2]:
# https://splinter.readthedocs.io/en/latest/drivers/chrome.html
!which chromedriver

/usr/local/bin/chromedriver


In [3]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [4]:
url = 'https://www.psychologytoday.com/us/therapists/california'
browser.visit(url)

In [5]:
#Empty list to store individual psychologists url
scrape_list = []

# Takes 3:30 sec for 500 pages (california)
for x in range(5):
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    links = soup.find_all('div', class_="col-4 col-sm-4 col-md-2 col-lg-2 photo-column")
    
    for link in links:
        link = link.find('a')
        href = str(link['href'])
        scrape_list.append(href)
        
    try:
        browser.find_by_css('.btn-next')[-1].click()
    except ElementDoesNotExist:
        print('Scraping Done!')
        break


In [6]:
scrape_list = list(set(scrape_list))
len(scrape_list)
# df = pd.DataFrame(columns=[['id', 'photo url', 'name', 'coordinates', 'website', 'phone number', 'specialties', 'issues', 'finances', 'treatment approaches', 'experience']])

56

In [7]:
item_id = []
photos = []
psychologists = []
coordinates=[]
psychologist_websites=[]
phone_numbers=[] 
specialties =[]
issues=[]
finances=[]
treatment_approaches=[]
experiences=[]
for list_item in range(len(scrape_list)):
    url = scrape_list[list_item]
    browser.visit(url)
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    item_id.append(list_item)
    
    # Scraping profile picture
    try:
        picture = soup.find('div', class_="col-4 col-sm-4 col-md-12 col-lg-12")
        picture_link = picture.find('img')
        picture_url = picture_link['src']
        if picture_url:
            photos.append(picture_url)
        else: 
            photos.append('NaN')
    except AttributeError:
        photos.append('NaN')
    try:
        psychologists_name= soup.find('div', class_="col-12 col-sm-12 hidden-md-up profile-title-container")
        psychologist_name=psychologists_name.find('h1').text
        psychologist_name = psychologist_name.strip('\n')
        psychologist_name = psychologist_name.lstrip()
        if psychologist_name:
            psychologists.append(psychologist_name)
        else: 
            psychologists.append('NaN')
    except AttributeError:
        psychologists.append('NaN')
    try:
        address = soup.find('div', class_="address-data")
        lat = address['data-map-lat']
        lon = address['data-map-lon']
        coordinate = [lat,lon]
        if coordinate:
            coordinates.append(coordinate)
        else: 
            coordinates.append('NaN')
    except (TypeError, KeyError):
        coordinates.append('NaN')
    try:
        phone= soup.find('a', class_="phone-number").text
        if phone:
            phone_numbers.append(phone)
        else:
            phone_numbers.append('NaN')
    except AttributeError:
        phone_numbers.append('NaN')
    psychologist_website = soup.find('a', class_='btn btn-md btn-profile btn-default hidden-sm-down')
    if psychologist_website:
        psychologist_website = psychologist_website['href']
        psychologist_websites.append(psychologist_website)
    else:
        psychologist_websites.append('NaN')
    
    specialties_list = soup.find_all('ul', class_='attribute-list specialties-list')
    specialties_group = []
    for specialty in specialties_list:
        items = specialty.find_all('li')
        for text in items:
            if text:
                text = text.text
                text = text.strip('\n')
                text = text.strip()
                specialties_group.append(text)
            else: 
                specialties_group.append('NaN')
    specialties.append(specialties_group)
               
            
    issues_list = soup.find_all('div', class_='spec-list attributes-issues')
    issues_group = []
    for issue in issues_list:
        items = issue.find_all('li')
        for item in items:
            if item:
                item = item.text
                item = item.strip('\n')
                item = item.lstrip()
                item = item.rstrip()
                issues_group.append(item)
            else: 
                issues_group.append('NaN')
    issues.append(issues_group)
    finance = soup.find_all('div', class_='profile-finances details-section top-border')
    try:
        for cost in finance:
            cost = cost.find('li')
            cost = cost.text
            cost_text = cost.split(':')[0]
            cost_text = cost_text.strip('\n')
            cost_text = cost_text.lstrip()
            cost_text = cost_text.rstrip()
            if cost_text == 'Avg Cost (per session)':
                cost = cost.split(':')[1]
                cost = cost.strip('\n')
                cost = cost.lstrip()
                cost = cost.rstrip()
                finances.append(cost)
            else:
                finances.append('NaN')
    except (IndexError, AttributeError):
        finances.append('NaN')
    experiences_years = soup.find_all('div', class_='profile-qualifications details-section') 
    try:
        for experience in experiences_years:
            years = experience.find('li')
            years = years.text
            years_text = years.split(':')[0]
            years_text = years_text.strip('\n')
            years_text = years_text.lstrip()
            years_text = years_text.rstrip()
            if years_text == 'Years in Practice':
                years = years.split(':')[1]
                years = years.strip('\n')
                years = years.lstrip()
                years = years.rstrip()
                experiences.append(years)
            else:
                experiences.append('NaN')
    except AttributeError:
        experiences.append('NaN')
        
    treatment_approaches_group = []
    treatment_qualifications = soup.find_all('div', class_='spec-list attributes-treatment-orientation')
    for treatment in treatment_qualifications:
        approaches = treatment.find_all('li')
        for approach in approaches:
            if approach:
                approach = approach.text
                approach = approach.strip('\n')
                approach = approach.strip()
                treatment_approaches_group.append(approach)
            else:
                treatment_approaches_group.append('NaN')
    treatment_approaches.append(treatment_approaches_group)

In [14]:
# len(item_id)
# photos
# psychologists
# coordinates
# psychologist_websites
len(phone_numbers)
# len(specialties)
# len(issues)
# len(finances)
# len(treatment_approaches)



56

In [25]:
ca_psy_info ={#"item_id" : item_id,
              "photos":photos,
              "psychologists name":psychologists,
              "coordinates":coordinates,
              "psychologist_websites":psychologist_websites,
              "phone_numbers":phone_numbers,
              "specialties":specialties,
              "issues":issues,
              "finances":finances,
              "treatment_approaches":treatment_approaches,
             }
ca_psy_info

{'photos': ['https://cdn4.sussexdirectories.com/rms/rms_photos/sized/38/65/386538-1193630-2_320x400.jpg?pu=1521956221',
  'https://cdn2.sussexdirectories.com/rms/rms_photos/sized/24/30/53024-547106-3_320x400.jpg?pu=1471561112',
  'https://cdn4.sussexdirectories.com/rms/rms_photos/sized/78/82/278278-575295-3_320x400.jpg?pu=1478797712',
  'https://cdn4.sussexdirectories.com/rms/rms_photos/sized/43/06/40643_2_120x150.jpg?pu=0',
  'https://cdn4.sussexdirectories.com/rms/rms_photos/sized/23/68/366823-1148859-2_320x400.jpg?pu=1516067719',
  'https://cdn2.sussexdirectories.com/rms/rms_photos/sized/09/71/117109-1208370-3_320x400.jpg?pu=1524036263',
  'https://cdn4.sussexdirectories.com/rms/rms_photos/sized/60/98/229860-1159891-1_320x400.jpg?pu=1517500691',
  'https://cdn4.sussexdirectories.com/rms/rms_photos/sized/48/08/70848_4_120x150.jpg?pu=0',
  'https://cdn2.sussexdirectories.com/rms/rms_photos/sized/43/15/401543-1306653-2_320x400.jpg?pu=1536515599',
  'https://cdn2.sussexdirectories.com/r

In [27]:
df = pd.DataFrame(columns=[['photo url', 'name', 'coordinates', 'website', 'phone number', 'specialties', 'issues', 'finances', 'treatment approaches']])
df

,photo url,name,coordinates,website,phone number,specialties,issues,finances,treatment approaches


In [30]:
ca_psy=pd.DataFrame.from_dict(ca_psy_info)
ca_psy.head()
len(ca_psy)

56

In [31]:
ca_psy=ca_psy.dropna()

In [33]:
len(ca_psy)
ca_psy

,coordinates,finances,issues,phone_numbers,photos,psychologist_websites,psychologists name,specialties,treatment_approaches
0,"[34.1077293, -118.2632832]",$140 - $180,"[Chronic Illness, Coping Skills, Parenting, Sl...",(818) 423-5039,https://cdn4.sussexdirectories.com/rms/rms_pho...,https://out.psychologytoday.com/us/profile/386...,Dana Grip,"[Relationship Issues, Anxiety, Depression]","[Cognitive Behavioral (CBT), Interpersonal, Mi..."
1,"[33.6722533, -117.8606614]",$100 - $140,"[Addiction, Alcohol Abuse, Anxiety, Behavioral...",(714) 794-5819,https://cdn2.sussexdirectories.com/rms/rms_pho...,https://out.psychologytoday.com/us/profile/530...,James Swaniger,"[Relationship Issues, Men's Issues, Women's Is...","[Coaching, Cognitive Behavioral (CBT), Existen..."
2,"[34.4253101, -119.6989451]",$100 - $150,"[Anger Management, Anxiety, Behavioral Issues,...",(818) 423-4037,https://cdn4.sussexdirectories.com/rms/rms_pho...,NaN,Robbie Culp,"[Eating Disorders, Depression, Self Esteem]","[Culturally Sensitive, Feminist, Humanistic]"
3,"[34.03951, -118.423311]",$60 - $80,"[Addiction, Anger Management, Depression, Divo...",(310) 895-9395,https://cdn4.sussexdirectories.com/rms/rms_pho...,NaN,Carol Kennedy,"[Relationship Issues, Anxiety, Life Coaching]","[Eclectic, Family / Marital, Interpersonal, Ps..."
4,"[34.1477408, -118.1310224]",$120 - $150,"[Anxiety, Behavioral Issues, Coping Skills, De...",(818) 465-5831,https://cdn4.sussexdirectories.com/rms/rms_pho...,NaN,"Harvey P. Shacket, LCSW",[Adjustment to Health Issues],"[Cognitive Behavioral (CBT), Compassion Focuse..."
5,"[34.0607258, -118.3002599]",$140 - $180,"[ADHD, Addiction, Anger Management, Bipolar Di...",(909) 667-7263,https://cdn2.sussexdirectories.com/rms/rms_pho...,https://out.psychologytoday.com/us/profile/117...,Debi Kim,"[Depression, Anxiety, Trauma and PTSD]","[Attachment-based, Brain-Based Therapy, Cognit..."
6,"[33.757873, -118.110964]",$150 - $180,"[Addiction, Alcohol Abuse, Career Counseling, ...",(657) 237-2117,https://cdn4.sussexdirectories.com/rms/rms_pho...,NaN,Donald J la Perna,"[Relationship Issues, Self Esteem, Marital and...","[Acceptance and Commitment Therapy (ACT), Cogn..."
7,"[34.062109, -118.383611]",$150 - $200,"[Addiction, Anxiety, Chronic Pain, Depression,...",(424) 625-3687,https://cdn4.sussexdirectories.com/rms/rms_pho...,https://out.psychologytoday.com/us/profile/708...,Christopher Emerson,"[Spirituality, Relationship Issues]","[Existential, Psychoanalytic, Psychodynamic, R..."
8,"[38.8525431, -120.0210448]",$120 - $150,"[ADHD, Adoption, Anger Management, Anxiety, Be...",(559) 785-3859,https://cdn2.sussexdirectories.com/rms/rms_pho...,https://out.psychologytoday.com/us/profile/401...,Christina Rodriguez,"[Child or Adolescent, Depression, Trauma and P...","[Art Therapy, Christian Counseling, Coaching, ..."
9,"[34.0768706, -118.3240085]",NaN,"[Coping Skills, Depression, Divorce, Family Co...",(213) 279-6330,https://cdn2.sussexdirectories.com/rms/rms_pho...,https://out.psychologytoday.com/us/profile/145...,Dimitry Granovsky,"[Relationship Issues, Anxiety, Addiction]","[Attachment-based, Family / Marital, Humanisti..."


In [ ]:
url = 'https://www.psychologytoday.com/us/therapists/maria-miller-concord-ca/49919?sid=1537065769.1125_27220&state=CA&ref=1&tr=ResultsPhoto'
browser.visit(url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

specialties=[]
specialties_list = soup.find_all('ul', class_='attribute-list specialties-list')
for specialty in specialties_list:
    items = specialty.find_all('li')
    group = [items]
    for item in items:
        if item:
            item = item.text
            item = item.strip('\n')
            item = item.lstrip()
            item = item.rstrip()
            group.append(item)
            
#             specialties.append(item)
        else: 
            specialties.append('NaN')
#         specialties=[specialties]
        specialties.append(item)
    specialties=[specialties]

specialties
# specialties
# qual= soup.find_all('div', class_="profile-qualifications details-section")
# qual1= qaul.find('div', class_="profile-qualifications details-section")
# print(qual)


In [ ]:
ca_psy_info ={"item_id" : item_id,
              "photos":photos,
              "psychologists":psychologists,
              "coordinates":coordinates,
              "psychologist_websites":psychologist_websites,
              "phone_numbers":phone_numbers,
              "websites":websites,
              "specialties":specialties,
              "issues":issues,
              "finances":finances,
              "treatment_approaches":treatment_approaches,
              "experiences":experiences
             }

ca_psy_info

In [1]:
from pymongo import MongoClient
import pandas as pd
import os
client = MongoClient()
db=client.therapists
# employee = db.hour
df = pd.read_csv("ca_therapists.csv") #csv file which you want to import
records_ = df.to_dict(orient = 'records')
result = db.ca_therapists.insert_many(records_ )

In [37]:
from flask import Flask, render_template, jsonify, redirect

In [38]:
jsonify(result)

RuntimeError: Working outside of application context.

This typically means that you attempted to use functionality that needed
to interface with the current application object in some way. To solve
this, set up an application context with app.app_context().  See the
documentation for more information.

In [ ]:
import pymongo
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)